In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

! pip install --upgrade pip
!pip install sentencepiece

In [2]:
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

In [3]:
train.describe(include="object")

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [4]:
train["input"] = "TEXT1: " + train.context + "; TEXT2: " + train.target + "; ANC1: " + train.anchor

In [5]:
train.head()

,id,anchor,target,context,score,input
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,TEXT1: A47; TEXT2: abatement of pollution; ANC...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,TEXT1: A47; TEXT2: forest region; ANC1: abatement


### Tokenization

In [6]:
!pip install datasets
from datasets import Dataset, DatasetDict

ds = Dataset.from_pandas(train)

In [7]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [8]:
model_nm = 'microsoft/deberta-v3-small'

In [9]:
!pip install transformers

from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
tokz.tokenize("Hi, I am Edgar and I love Machine Learning!")

['▁Hi',
 ',',
 '▁I',
 '▁am',
 '▁Edgar',
 '▁and',
 '▁I',
 '▁love',
 '▁Machine',
 '▁Learning',
 '!']

In [11]:
def tok_func(x): return tokz(x["input"])

In [12]:
tok_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [13]:
row = tok_ds[0]

row["input"], row["input_ids"]

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [33]:
tokz.vocab["▁Edgar"]

17520

In [15]:
tok_ds = tok_ds.rename_columns({"score" : "labels"})

In [16]:
eval_df = pd.read_csv("/content/test.csv")
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [17]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [18]:
eval_df["input"] = "TEXT1: " + eval_df.context + " ; TEXT2: " + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [19]:
from transformers import TrainingArguments, Trainer

In [20]:
bs = 128
epochs =4

In [21]:
lr = 8e-5

In [22]:
! pip install -U accelerate
! pip install -U transformers

import accelerate
import transformers

transformers.__version__, accelerate.__version__


args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')


In [23]:
def corr(x,y): return np.corrcoef(x,y)[0][1]

In [24]:
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [25]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_nm, num_labels=1
)

trainer = Trainer(model, args, train_dataset=dds["train"],
                  eval_dataset=dds["test"],
                  tokenizer=tokz, compute_metrics= corr_d)

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [26]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.025323,0.798335
2,No log,0.025353,0.821894
3,0.036900,0.022942,0.830699
4,0.036900,0.022726,0.831368


TrainOutput(global_step=856, training_loss=0.027043238978519617, metrics={'train_runtime': 213.4425, 'train_samples_per_second': 512.625, 'train_steps_per_second': 4.01, 'total_flos': 716605488222960.0, 'train_loss': 0.027043238978519617, 'epoch': 4.0})

In [27]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[ 0.51806641],
       [ 0.70654297],
       [ 0.55957031],
       [ 0.37207031],
       [ 0.03015137],
       [ 0.55224609],
       [ 0.5078125 ],
       [-0.03811646],
       [ 0.28564453],
       [ 1.0859375 ],
       [ 0.27172852],
       [ 0.26391602],
       [ 0.75439453],
       [ 0.83740234],
       [ 0.71826172],
       [ 0.5234375 ],
       [ 0.32250977],
       [-0.04318237],
       [ 0.68212891],
       [ 0.38476562],
       [ 0.46289062],
       [ 0.23535156],
       [ 0.12451172],
       [ 0.26391602],
       [ 0.61914062],
       [-0.01358795],
       [-0.03112793],
       [-0.00463104],
       [-0.02348328],
       [ 0.60986328],
       [ 0.32421875],
       [ 0.0073204 ],
       [ 0.70996094],
       [ 0.50585938],
       [ 0.44384766],
       [ 0.21398926]])

In [28]:
preds = np.clip(preds, 0, 1)

In [29]:
preds

array([[0.51806641],
       [0.70654297],
       [0.55957031],
       [0.37207031],
       [0.03015137],
       [0.55224609],
       [0.5078125 ],
       [0.        ],
       [0.28564453],
       [1.        ],
       [0.27172852],
       [0.26391602],
       [0.75439453],
       [0.83740234],
       [0.71826172],
       [0.5234375 ],
       [0.32250977],
       [0.        ],
       [0.68212891],
       [0.38476562],
       [0.46289062],
       [0.23535156],
       [0.12451172],
       [0.26391602],
       [0.61914062],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.60986328],
       [0.32421875],
       [0.0073204 ],
       [0.70996094],
       [0.50585938],
       [0.44384766],
       [0.21398926]])

In [30]:
import datasets

submission = datasets.Dataset.from_dict({
    "id" : eval_ds["id"],
    "score" : preds
})

submission.to_csv("submission.csv", index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1030